# PRACTICA 3 - GRUPO 5

Para el desarrollo de esta práctica hemos seleccionado de kaggle el dataset "Global Cybersecurity Threats", el mismo que posee un registro de información sobre cyberattacks, malware, industrias y países afectados, entre otros tipos de información relacionada, la cual procederemos a analizar.

## Tratamiento y Preprocesamiento

Como primer paso realizaremos la instalación de las librerías que utilizaremos durante la práctica.


In [1]:
#Instalación de los paquetes/modelos
%pip install NLTK
%pip install transformers
%pip install spaCy
!python -m spacy download en_core_web_sm

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.8/31.8 MB 35.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 29.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 29.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 20.8 MB/s eta 0:00:0000:010:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [12]:
# Importamos la librería pandas, numpy, matplotlib y seaborn
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

Vamos a extraer el archivo .csv de la ruta DATA.

In [3]:
# Ruta de acceso a los datos
csv_path = 'DATA/Global_Cybersecurity_Threats_2015-2024.csv'

# Especificamos el separador
data = pd.read_csv(csv_path, sep=',')

# Mostrar las primero 5 filas
data.head()

,Country,Year,Attack Type,Target Industry,Financial Loss (in Million $),Number of Affected Users,Attack Source,Security Vulnerability Type,Defense Mechanism Used,Incident Resolution Time (in Hours)
0,China,2019,Phishing,Education,80.53,773169,Hacker Group,Unpatched Software,VPN,63
1,China,2019,Ransomware,Retail,62.19,295961,Hacker Group,Unpatched Software,Firewall,71
2,India,2017,Man-in-the-Middle,IT,38.65,605895,Hacker Group,Weak Passwords,VPN,20
3,UK,2024,Ransomware,Telecommunications,41.44,659320,Nation-state,Social Engineering,AI-based Detection,7
4,Germany,2018,Man-in-the-Middle,IT,74.41,810682,Insider,Social Engineering,VPN,68


Como parte del procesamiento vamos a identificar el tamaño de la DATA.

In [6]:
print(data.shape)


(3000, 10)


Ahora llevaremos toda la información tipo texto de la tabla a minúsculas, para ello usaremos la función .apply y lambda.col para recorrer las columnas de la data en busca de texto, una vez identificada, se procede a convertir a minúsculas con .strlower().

In [ ]:
# Convertir todo el Data tipo texto a minúsculas
data = data.apply(lambda col: col.str.lower() if col.dtype == 'object' else col)

# Verificamos los cambios
print(data.head())

   Country  Year        Attack Type     Target Industry  \
0    china  2019           phishing           education   
1    china  2019         ransomware              retail   
2    india  2017  man-in-the-middle                  it   
3       uk  2024         ransomware  telecommunications   
4  germany  2018  man-in-the-middle                  it   

   Financial Loss (in Million $)  Number of Affected Users Attack Source  \
0                          80.53                    773169  hacker group   
1                          62.19                    295961  hacker group   
2                          38.65                    605895  hacker group   
3                          41.44                    659320  nation-state   
4                          74.41                    810682       insider   

  Security Vulnerability Type Defense Mechanism Used  \
0          unpatched software                    vpn   
1          unpatched software               firewall   
2              weak 

También verificamos si existen valores faltantes en alguna de las columas, para esto usamos .isnull() y .sum() para contabilizar.

In [9]:
# Vrificamos si existen valores faltantes
valores_faltantes = data.isnull().sum()
print('Valores faltantes por columna:')
print(valores_faltantes)

Valores faltantes por columna:
Country                                0
Year                                   0
Attack Type                            0
Target Industry                        0
Financial Loss (in Million $)          0
Number of Affected Users               0
Attack Source                          0
Security Vulnerability Type            0
Defense Mechanism Used                 0
Incident Resolution Time (in Hours)    0
dtype: int64


En caso de existir datos vacíos procedemos a rellenar con la palabra "faltante". Utilizamos la función .apply y lambda.x para buscar en cada celda de una columna y con  isinstance(x, str) identificamos si es del tipo texto, luego con x.strip() eliminamos los espacios vacíos. 

In [11]:
# Rellenar los espacios en blanco con un valor específico ('faltante')
data = data.apply(lambda x: 'faltante' if isinstance(x, str) and (x.strip() == '' or x is None) else x)

# Mostrar la Data después de rellenar
print("\nData después de rellenar los espacios en blanco:")
print(data)


Data después de rellenar los espacios en blanco:
      Country  Year        Attack Type     Target Industry  \
0       china  2019           phishing           education   
1       china  2019         ransomware              retail   
2       india  2017  man-in-the-middle                  it   
3          uk  2024         ransomware  telecommunications   
4     germany  2018  man-in-the-middle                  it   
...       ...   ...                ...                 ...   
2995       uk  2021         ransomware          government   
2996   brazil  2023      sql injection  telecommunications   
2997   brazil  2017      sql injection                  it   
2998       uk  2022      sql injection                  it   
2999  germany  2021      sql injection              retail   

      Financial Loss (in Million $)  Number of Affected Users Attack Source  \
0                             80.53                    773169  hacker group   
1                             62.19            